In [50]:
# 数据处理
import numpy as np
import pandas as pd

#绘图
import seaborn as sns
import matplotlib.pyplot as plt

#各种模型、数据处理方法
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
import warnings
warnings.filterwarnings('ignore')

In [51]:
'''数据读入'''
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine_df = pd.concat([train_df, test_df])

In [52]:
combine_df['NameSize'] = combine_df['Name'].apply(lambda x: len(x))
# cut将根据值本身来选择箱子均匀间隔，qcut是根据这些值的频率来选择箱子的均匀间隔
combine_df['NameSize'] = pd.qcut(combine_df['NameSize'],5) # 分箱
# combine_df.groupby(combine_df['Name'].apply(lambda x: x.split(', ')[1]).apply(lambda x: x.split('.')[0]))['Survived'].mean().plot()
# plt.show()
titles = combine_df['Name'].apply(lambda x: x.split(', ')[1]).apply(lambda x: x.split('.')[0])
combine_df['Title'] = titles
combine_df['Title'] = combine_df['Title'].replace(['Don','Dona', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir','Dr', 'Master'],'Mr')
combine_df['Title'] = combine_df['Title'].replace(['Mlle','Ms'], 'Miss')
combine_df['Title'] = combine_df['Title'].replace(['the Countess','Mme','Lady','Dr'], 'Mrs')
df = pd.get_dummies(combine_df['Title'],prefix='Title')
combine_df = pd.concat([combine_df,df],axis=1)

In [53]:
combine_df['Fname'] = combine_df['Name'].apply(lambda x:x.split(',')[0])
combine_df['FamilySize'] = combine_df['SibSp']+combine_df['Parch']
dead_female_Fname = list(set(combine_df[(combine_df.Sex=='female') & (combine_df.Age>=12)
                              & (combine_df.Survived==0) & (combine_df.FamilySize>1)]['Fname'].values))
survive_male_Fname = list(set(combine_df[(combine_df.Sex=='male') & (combine_df.Age>=12)
                              & (combine_df.Survived==1) & (combine_df.FamilySize>1)]['Fname'].values))
combine_df['Dead_female_family'] = np.where(combine_df['Fname'].isin(dead_female_Fname),1,0)
combine_df['Survive_male_family'] = np.where(combine_df['Fname'].isin(survive_male_Fname),1,0)
combine_df = combine_df.drop(['Name','Fname'],axis=1)
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 19 columns):
Age                    1046 non-null float64
Cabin                  295 non-null object
Embarked               1307 non-null object
Fare                   1308 non-null float64
Parch                  1309 non-null int64
PassengerId            1309 non-null int64
Pclass                 1309 non-null int64
Sex                    1309 non-null object
SibSp                  1309 non-null int64
Survived               891 non-null float64
Ticket                 1309 non-null object
NameSize               1309 non-null category
Title                  1309 non-null object
Title_Miss             1309 non-null uint8
Title_Mr               1309 non-null uint8
Title_Mrs              1309 non-null uint8
FamilySize             1309 non-null int64
Dead_female_family     1309 non-null int64
Survive_male_family    1309 non-null int64
dtypes: category(1), float64(3), int64(7), object(5), uint8(3)
me

In [54]:
group = combine_df.groupby(['Title', 'Pclass'])['Age']
combine_df['Age'] = group.transform(lambda x: x.fillna(x.median()))
combine_df = combine_df.drop('Title', axis = 1)
combine_df['IsChild'] = np.where(combine_df['Age'] <= 12,1,0) # 如果小于12岁则标记为1，否则0
combine_df['Age'] = pd.cut(combine_df['Age'],5)
combine_df = combine_df.drop('Age',axis=1)

In [55]:
combine_df['FamilySize'] = np.where(combine_df['FamilySize'] == 0, 'solo',
                                    np.where(combine_df['FamilySize'] <= 3, 'normal','big'))
df = pd.get_dummies(combine_df['FamilySize'], prefix='FamilySize')
combine_df = pd.concat([combine_df, df], axis=1).drop(['SibSp', 'Parch', 'FamilySize'], axis=1)
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 18 columns):
Cabin                  295 non-null object
Embarked               1307 non-null object
Fare                   1308 non-null float64
PassengerId            1309 non-null int64
Pclass                 1309 non-null int64
Sex                    1309 non-null object
Survived               891 non-null float64
Ticket                 1309 non-null object
NameSize               1309 non-null category
Title_Miss             1309 non-null uint8
Title_Mr               1309 non-null uint8
Title_Mrs              1309 non-null uint8
Dead_female_family     1309 non-null int64
Survive_male_family    1309 non-null int64
IsChild                1309 non-null int64
FamilySize_big         1309 non-null uint8
FamilySize_normal      1309 non-null uint8
FamilySize_solo        1309 non-null uint8
dtypes: category(1), float64(2), int64(5), object(4), uint8(6)
memory usage: 131.7+ KB
None


In [56]:
combine_df['Ticket_Lett'] = combine_df['Ticket'].apply(lambda x: str(x)[0])
# combine_df['Ticket_Lett'] = combine_df['Ticket_Lett'].apply(lambda x: str(x))

combine_df['High_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['1', '2', 'P']),1,0)
combine_df['Low_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['A','W','3','7']),1,0)
combine_df = combine_df.drop(['Ticket','Ticket_Lett'],axis=1)
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 19 columns):
Cabin                   295 non-null object
Embarked                1307 non-null object
Fare                    1308 non-null float64
PassengerId             1309 non-null int64
Pclass                  1309 non-null int64
Sex                     1309 non-null object
Survived                891 non-null float64
NameSize                1309 non-null category
Title_Miss              1309 non-null uint8
Title_Mr                1309 non-null uint8
Title_Mrs               1309 non-null uint8
Dead_female_family      1309 non-null int64
Survive_male_family     1309 non-null int64
IsChild                 1309 non-null int64
FamilySize_big          1309 non-null uint8
FamilySize_normal       1309 non-null uint8
FamilySize_solo         1309 non-null uint8
High_Survival_Ticket    1309 non-null int64
Low_Survival_Ticket     1309 non-null int64
dtypes: category(1), float64(2), int64(7), object(

In [46]:
combine_df['Ticket_Lett'] = combine_df['Ticket'].apply(lambda x: str(x)[0])
# combine_df['Ticket_Lett'] = combine_df['Ticket_Lett'].apply(lambda x: str(x))
combine_df['High_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['1', '2', 'P']),1,0)
combine_df['Low_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['A','W','3','7']),1,0)
combine_df = combine_df.drop(['Ticket','Ticket_Lett'],axis=1)
print(combine_df.info())

KeyError: 'Ticket'

In [57]:
combine_df.Embarked = combine_df.Embarked.fillna('S')
df = pd.get_dummies(combine_df['Embarked'],prefix='Embarked')
combine_df = pd.concat([combine_df,df],axis=1).drop('Embarked',axis=1)
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 21 columns):
Cabin                   295 non-null object
Fare                    1308 non-null float64
PassengerId             1309 non-null int64
Pclass                  1309 non-null int64
Sex                     1309 non-null object
Survived                891 non-null float64
NameSize                1309 non-null category
Title_Miss              1309 non-null uint8
Title_Mr                1309 non-null uint8
Title_Mrs               1309 non-null uint8
Dead_female_family      1309 non-null int64
Survive_male_family     1309 non-null int64
IsChild                 1309 non-null int64
FamilySize_big          1309 non-null uint8
FamilySize_normal       1309 non-null uint8
FamilySize_solo         1309 non-null uint8
High_Survival_Ticket    1309 non-null int64
Low_Survival_Ticket     1309 non-null int64
Embarked_C              1309 non-null uint8
Embarked_Q              1309 non-null uint8
Embarke

In [58]:
combine_df['Cabin_isNull'] = np.where(combine_df['Cabin'].isnull(), 0, 1)
combine_df = combine_df.drop('Cabin', axis=1)
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 21 columns):
Fare                    1308 non-null float64
PassengerId             1309 non-null int64
Pclass                  1309 non-null int64
Sex                     1309 non-null object
Survived                891 non-null float64
NameSize                1309 non-null category
Title_Miss              1309 non-null uint8
Title_Mr                1309 non-null uint8
Title_Mrs               1309 non-null uint8
Dead_female_family      1309 non-null int64
Survive_male_family     1309 non-null int64
IsChild                 1309 non-null int64
FamilySize_big          1309 non-null uint8
FamilySize_normal       1309 non-null uint8
FamilySize_solo         1309 non-null uint8
High_Survival_Ticket    1309 non-null int64
Low_Survival_Ticket     1309 non-null int64
Embarked_C              1309 non-null uint8
Embarked_Q              1309 non-null uint8
Embarked_S              1309 non-null uint8
Cabin_i

In [59]:
df = pd.get_dummies(combine_df['Pclass'], prefix='Pclass')
combine_df = pd.concat([combine_df, df], axis=1).drop('Pclass', axis=1)


In [60]:
df = pd.get_dummies(combine_df['Sex'], prefix='Sex')
combine_df = pd.concat([combine_df, df], axis=1).drop('Sex',axis=1)


In [61]:
combine_df['Fare'].fillna(combine_df['Fare'].mode()[0], inplace=True)
# print(combine_df['Fare'].value_counts())
combine_df['Fare'] = pd.qcut(combine_df['Fare'], 3)
df = pd.get_dummies(combine_df['Fare'], prefix='Fare').drop('Fare_(-0.001, 8.662]',axis=1)
# print(combine_df['Fare'].value_counts())
combine_df = pd.concat([combine_df, df], axis=1).drop('Fare', axis=1)
# print(combine_df.info())

In [62]:
features = combine_df.drop(['PassengerId', 'Survived'], axis=1).columns
# print(features)
le = LabelEncoder()
for feature in features:
    le = le.fit(combine_df[feature])
    combine_df[feature] = le.transform(combine_df[feature])

In [63]:
X_all = combine_df.iloc[:891,:].drop(['PassengerId', 'Survived'], axis=1)
Y_all = combine_df.iloc[:891,:]['Survived']
X_test = combine_df.iloc[891:,:].drop(['PassengerId', 'Survived'], axis=1)
# print(X_all.iloc[0,:])

In [64]:
class Ensemble(object):

    def __init__(self, estimators):
        self.estimator_names = []
        self.estimators = []
        for i in estimators:
            self.estimator_names.append(i[0])
            self.estimators.append(i[1])
        self.clf = LogisticRegression()

    def fit(self, x_train, y_train):
        for i in self.estimators:
            i.fit(x_train, y_train)
        x = np.array([i.predict(x_train) for i in self.estimators]).T
        y = y_train
        self.clf.fit(x, y)

    def predict(self, x):
        x = np.array([i.predict(x) for i in self.estimators]).T
        return self.clf.predict(x)

    def score(self, x, y):
        s = precision_score(y, self.predict(x))
        return s


In [65]:
lr = LogisticRegression()
svc = SVC()
rf = RandomForestClassifier(n_estimators=300,min_samples_leaf=4,class_weight={0:0.745,1:0.255})
gbdt = GradientBoostingClassifier(n_estimators=500,learning_rate=0.03,max_depth=3)
xgbt = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.03)


In [71]:
ensemble = Ensemble([('lr',lr),('rf',rf),('svc',svc),('gbdt',gbdt),('xgbt',xgbt)])
score = 0
for i in range(0, 10):
    num_test = 0.20
    x_train, x_cv, y_train, y_cv = train_test_split(X_all.values, Y_all.values, test_size=num_test)
    ensemble.fit(x_train, y_train)
#     # Y_test = ensemble.predict(X_test)
    acc = round(ensemble.score(x_cv, y_cv) * 100 , 2)
    score += acc
print(score/10)

82.71000000000001


In [72]:

ensemble.fit(X_all, Y_all)
Y_test = ensemble.predict(X_test).astype(int)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'],
                           'Survived': Y_test})
submission.to_csv('submission1.csv', index=False)

ValueError: feature_names may not contain [, ] or <